# MSCF 46982 Market Microstructure and Algorithmic Trading

Fall 2025 Mini 2

Getting Started with Q 

Sourced from [Q Tips: Fast, Scalable and Maintainable Kdb+][qtips]

Copyright &copy; 2025 Nick Psaris. All Rights Reserved

[qtips]: https://www.q-tips.net/ "Q Tips"


# TOC
- [Initialize](#Initialize)
- [Atoms and Lists](#Atoms-and-Lists)
- [12 Uniforms](#12-Uniforms)
- [No Stinkin Loops](#No-Stinkin-Loops)
- [Box-Muller Transform](#Box-Muller-Transform)
- [Geometric Brownian Motion](#Geometric-Brownian-Motion)
- [Newton-Raphson](#Newton-Raphson)
- [Temporal Data](#Temporal-Data)
- [Security Paths](#Security-Paths)
- [Dictionaries](#Dictionaries)
- [Tables](#Tables)
- [Keyed Tables](#Keyed-Tables)
- [Option Pricing](#Option-Pricing)
- [Monte Carlo Simulation](#Monte-Carlo-Simulation)
- [Attacker Success Probability](#Attacker-Success-Probability)

# Initialize
- We start by initializing PyKX in Q-first mode and the number of rows
  and columns displayed


In [ ]:
import os
os.environ['PYKX_JUPYTERQ'] = 'true'
os.environ['PYKX_4_1_ENABLED'] = 'true'
import pykx as kx


In [ ]:
\c 10 100


# Atoms and Lists

- Operators apply to atoms and lists intuitively

In [ ]:
1+1 2 3 4

- Q code is evaluated:
  + [dextrosinistral][] (where dexter and sinister mean "left" and
    "right" respectively in Latin)
  + from right **to** left
  + left **of** right as in 'f **of** g **of** h **of** x':
     `f[g[h[x]]]` or equivalently `f g h x`
- The `til` operator generates a list of incremental integers

[dextrosinistral]: https://en.wiktionary.org/wiki/dextrosinistral "Dextrosinistral"

In [ ]:
1+til 10

- Division is performed with the `%` operator
- Infinities look like `0w`

In [ ]:
1%til 10 

- The `?` operator can be used to generate uniform random variates

In [ ]:
/ with replacement
10?10
/ or without
-10?10

- The `\S` system command displays the pseudorandom number generator
  (PRNG) seed (which defaults to the first 6 digits of $\pi$)
- The `\S` system command can also be used to set the PRNG seed
- Resetting the PRNG permits reproducible *random* computations


In [ ]:
\S

In [ ]:
\S -314159

# [12 Uniforms][u12]
- Comments start with `/`
- Assignment is performed with `:`
- Equality is tested with `=`
- Functions are enclosed by braces

[u12]: https://en.wikipedia.org/wiki/Irwin%E2%80%93Hall_distribution#Approximating_a_Normal_distribution
"Approximating a Normal distribution"

In [ ]:
/ 12 uniforms
u12:{-6+sum 12?1f} / this is also a comment

In [ ]:
u12[]

# [No Stinkin Loops][nsl]
- Loops are written as vector transformations
- The `0N!` operator prints values with no side effects
- Some functions are not 'atomic' and therefore need to iterated
- Monadic functions can be iterated with the `each` adverb (sometimes
  referred to as an iterator)

[nsl]: http://nsl.com/ "No Stinkin Loops"


In [ ]:
u12 each 0N!til 5

In [ ]:
n:1000000
x:u12 each til n
avg x
sdev x

# [Box-Muller Transform][]
- Functions have automatic x,y,z variables
- Multi-line functions **must** be indented
- Statements end with semicolons
- `if` statements do not have else clauses
- Modulus is performed with `mod` operator
- Exceptions are thrown with ``'`message`` or ``'"long message"``
- The `#` operator reshapes a list
- **Compound assignment** is performed with operator-colon
- Functions return the last evaluated expression

[Box-Muller Transform]: https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform
"Box-Muller Transform"

In [ ]:
/ box-muller
bm:{
 if[count[x] mod 2;'`length];
 x:2 0N#x;
 r:sqrt -2f*log first x;
 theta:2f*acos[-1f]*last x;
 x: r*cos theta;
 x,:r*sin theta;
 x}

## Dyadic adverbs
- Adverbs can be applied to any dyadic function
  - `'`  each-both
  - `':` each-prior
  - `\:` each-left
  - `/:` each-right
- Arguments are passed with brackets or juxtaposition
- Juxtaposition is the same as the apply operator `@`

In [ ]:
bm 4?1f
bm[4?1f]
(avg;dev) @\: bm 100000?1f

## Function timing
- The `\t` and `\ts` system commands profile a function for time (and space)

In [ ]:
\ts u12 each til n
\ts bm n?1f

# [Geometric Brownian Motion][gbm]

$$
S_{t}=S_0 e^{t(r-.5\sigma^2)+z\sigma\sqrt t}
$$

Function arguments are:
- Surrounded by brackets
- Separated by semicolons
- Limited to 8

[gbm]: https://en.wikipedia.org/wiki/Geometric_Brownian_motion
"Geometric Brownian motion"

In [ ]:
/ geometric brownian motion
/ (s)igma, (r)ate, (t)ime, z:normal random
/ user multiplies by (S)pot
gbm:{[s;r;t;z]exp (t*r-.5*s*s)+z*s*sqrt t}

In [ ]:
/ one step of a random walk
100*gbm[.30;.05;1%365.25] first bm 2?1f

## A random walk
We can accumulate random walk steps using the running-product operator
`prds`


In [ ]:
/ one year of random walks
100*prds gbm[.30;.05;1%365.25] bm 252?1f

- You can see see the `k` implementation of many operators

In [ ]:
prds

- You can write `k` code by prefixing with `k)`

In [ ]:
k)*\ 1+!10

## Monadic adverbs
- In `k`, `prd` and `prds` are defined in terms of the `over` and `scan` adverbs (iterators)
- The `over` adverb `/` iterates but only returns the last value (reduce)
- The `scan` adverb `\` iterates and returns all the intermediate values (accumulate)

In [ ]:
/ naive (memory inefficient) factorial
k)*/1+!10

- Operators with `scan` and `over` variants:
  - `prds` vs `prd`
  - `maxs` vs `max`
  - `mins` vs `min`
  - `sums` vs `sum`

# [Newton-Raphson][nr]
We can use the converging Newton-Raphson algorithm to find the root of a function
$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
$$
- if [else if [else if] ... ] else statements are performed with the `$[;;]` operator

[nr]: https://en.wikipedia.org/wiki/Newton%27s_method "Newton's method"

In [ ]:
/ newton-raphson
/ (e)rror tolerance, (f)unction that returns f(x) and f'(x)
nr:{[e;f;x]$[e>abs d:first[r]%last r:f x;x;x-d]}
/ (f;f') for y = -2+x^2
f:{(-2+x*x;2*x)}

The `scan` and `over` adverbs (iterators) are overloaded so that they can:
- Run until convergence or the initial values is seen again (if no
  left operand is provided). If the initial value is seen again, the
  value of the prior iteration is returned.
- Run a fixed number of times (if an integer is provided as the left
  operand)
- Run until a condition is met (if a function is provided as the left
  operand)


In [ ]:
/ convergence
\P 7
nr[.001;f] over 1f

In [ ]:
/ n iterations
2 nr[0;{(-2+x*x;2*x)}]\ 1f

In [ ]:
/ iterate until condition
{.1<abs sqrt[2]-x} nr[0;{(-2+x*x;2*x)}]\ 1f

# Temporal Data
- minute: `09:00`
- seconds: `09:00:00`
- time: `09:00:00.000`
- date: `2000.01.01`
- datetime: `2000.01.01T09:00:00.000` (date+time, deprecated)
- timespan: `0D09:00:00.000000000` (nanosecond precision)
- timestamp: `2001.01.01D09:00:00.000000000` (date+timespan)
- today: `.z.D` (`.z.d` for GMT)
- now: `.z.N` or `.z.T` or `.z.P` (`.z.n` or `.z.t` or `.z.p` for GMT)

# Security Paths
We first define a `rng` utility to generate a range of values

In [ ]:
/ return a range of numbers between (s)tart and (e)nd
/ with specified (w)indow sizep
rng:{[w;s;e]s+w*til 1+floor(e-s)%w}

show dts:rng[1;2000.01.01;2000.12.31]

Then we create a `path` function which uses the Geometric Brownian
Motion function `gbm` to generate random security paths with daily
time-steps


In [ ]:
/ generate simulated security paths
/ (s)igma, (r)ate, (t)ime
path:{[s;r;t]
 z:bm count[t]?1f;
 p:prds gbm[s;r;deltas[first t;t]] z;
 p}

show px:100*path[.3;.01] dts%365.25

# Dictionaries
- A dictionary is stored as two lists
- Lookups (by default) are iterations across the key, then the value is indexed
- A dictionary is also a list, and can be `reverse`ed, `each`ed, etc...
- The `#` operator takes n (-n) values from the beginning (end) of a list

In [ ]:
show -5#d:dts!px

## Dictionary keys and values
- The operators `key` and `value` grab the respective lists

In [ ]:
key d

In [ ]:
value d

## Indexing a dictionary
- A dictionary is indexed, assigned and joined like an array

In [ ]:
d[2000.01.01]:101f
d[2000.01.01]

In [ ]:
d,:(1#2000.01.01)!1#100f
d 2000.01.01

## New Dictionary Syntax
- Kdb+ 4.1 has introduced a new dictionary syntax
- A Dictionary can now be created with `([;])` syntax
- The keys must be symbols

In [ ]:
([date:dts;price:px])

# Tables
- **A table is a `flip`ped dictionary of lists**
- Strings are surrounded by quotation marks (and are allocate their own memory)
- Symbols are [interned strings][]
- Symbols are prefixed by a backtick (and all point to a unique copy of the text)

[interned strings]: https://en.wikipedia.org/wiki/String_interning "String interning"

In [ ]:
flip d:`date`price!m:(dts;px)

- Flipping a matrix actually reallocates memory
- Flipping a dictionary of lists merely sets a flag to indicate the
  data should be interepreted as a table


In [ ]:
\ts flip m / flip a matrix

In [ ]:
\ts flip d / flip a dictionary of lists

## List of conforming dictionaries
- A list of conforming dictionaries are (sometimes frustratingly)
  automatically converted into a table
- Iterating over a table (inefficiently) returns each row as a
  dictionary (similar to `pandas.DataFrame.iterrows()`)


In [ ]:
enlist `date`price!4 5

## Table literal
- A table can be created with `([];)` syntax
- Now we can see how Kdb+ works as a columnar database

In [ ]:
([]date:dts;price:px)

# Keyed Tables
- Keyed tables are (brilliantly) modeled as dictionaries where the key
  and value are both tables
- Keys are enclosed by brackets

In [ ]:
show t:([date:dts;sym:`BAC]price:px)

## Specifying keys
- Keys can be added and removed to tables with `n!t` syntax (where `n`
  indicates the number of key columns)
- Keys can also be created and removed with the `xkey` operator
  (useful when the column are not already at front)


In [ ]:
`sym`date xkey t

## Keyed tables are dictionaries
- The `key` and `value` operators work as expected

In [ ]:
key t

In [ ]:
value t

## Indexing keyed tables
- Single-column keys can be indexed with a single value
- Multi-column keys require indexing with a list
- Both single- and multi-column keys can be indexed with a dictionary
- Indexing for multiple records requires a table and logically returns a table

In [ ]:
t (2000.01.02;`BAC)

In [ ]:
t ([]date:2000.01.02 2000.01.03;sym:`BAC)

# Option Pricing
The closed form solution for the
[price](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#Black%E2%80%93Scholes_formula)
of calls/puts and each of the
[greeks](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#The_Options_Greeks)
requires the cumulative normal distribution.

\begin{align}
C(S_t,t) &= N(d_1)S_t - N(d_2)Ke^{-r(T-t)} \\
P(S_t,t) &= N(-d_2)Ke^{-r(T-t)} - N(-d_1)S_t \\
d_1 &= \frac{1}{\sigma\sqrt{T-t}}\left[ \ln\left(\frac{S_t}{K}\right)+\left(r+\frac{\sigma^2}{2}\right)(T-t) \right] \\
d_2 &= d_1 - \sigma\sqrt{T-t}
\end{align}

To implement the cumulative normal distribution, we use the Horner
rule to compute the error function $\text{erf}$.


## Horner Rule
- Used for efficient polynomial expansion
$$ a_0 + a_1x + a_2x^2 + a_3x^3 + \cdots + a_nx^x $$
$$ = a_0 + x(a_1 + x(a_2 + x(a_3 + \cdots + x(a_{n-1} + xa_n) \cdots ))) $$

In [ ]:
horner:{{z+y*x}[y]/[x]}

In [ ]:
horner[1 2 3 4;5]
4 + (3*5) + (2*5*5) + (1*5*5*5) 

## Error Function
- Can be approximated with a polynomial

For $x>=0$,

$$
\text{erf}(x) \approx 1 - (a_1t + a_2t^2 + \cdots + a_5t^5)e^{-x^2}, t = \frac{1}{1+px} 
$$
where
\begin{align}
  p &= 0.3275911      \\
  a_1 &= 0.254829592  \\
  a_2 &= −0.284496736 \\
  a_3 &= 1.421413741  \\
  a_4 &= −1.453152027 \\
  a_5 &= 1.061405429  \\
\end{align}

For $x<0$,

use $\text{erf}(x) = -\text{erf}(-x)$

## Error function implementation
- Normally provided in mathematical packages
- But we can implement it ourselves

In [ ]:
erf:{
 a:1.061405429 -1.453152027 1.421413741 -0.284496736 0.254829592;
 t:1f%1f+0.3275911*abs x;
 t:1f-t*horner[a;t]*exp neg x*x;
 x:t*1 -1f x<0f;
 x}

In [ ]:
erf -0w -.5 0 .5 0w

## Cumulative Normal 
- Can be written in terms of the Error Function
$$ \Phi  = \frac{1+\text{erf} (x/\sqrt{2})}{2}$$

In [ ]:
cnorm:{.5*1f+erf x%sqrt 2f}

In [ ]:
cnorm -0w -.5 0 .5 0w

## Black-Scholes-Merton
- Using all our building blocks, we implement an *atomic*
  Black-Scholes-Merton function.


In [ ]:
bsm:{[S;k;r;t;c;s]
 x:(log[S%k]+rt:r*t)%ssrt:s*srt:sqrt t;
 d1:ssrt+d2:x-.5*ssrt;
 n1:m*cnorm d1*m:-1 1f c; / boolean used for indexing
 n2:m*cnorm d2*m;
 p:(S*n1)-n2*pvk:k*pv:exp neg rt;
 g:(n1p:exp[-.5*d1*d1]%sqrt 2f*acos -1f)%S*ssrt;
 v:srt*Sn1p:S*n1p;
 th:neg (r*pvk*n2)+Sn1p*s*.5%srt;
 rho:pvk*t*n2;
 d:`price`delta`gamma`vega`theta`rho;
 d!:(p;n1;g;v;th;rho);
 if[0h<type p;d:flip d]; / distinguish between dictionary and table
 d}

## Atomic functions
- Atomic functions return an atom when passed an atom and return a
  list when passed a list
- We can treat a table as a list of dictionaries

In [ ]:
S:95f;k:100f;r:.01;t:.1;;c:1b;s:.3
bsm[S;k;r;t;c;s] / atomic evaluation

In [ ]:
([]spot)!bsm[;k;r;t;c;s] spot:S + til 10 / vector evaluation

# Monte Carlo Simulation
- Functional (as opposed to object oriented) programming separates
  data from algorithms
- Kdb+ provides the `peach` operator -- a parallel version of
  `each` -- thus allowing us to obtain immediate performance gains
  with a minor (one character) change of our code
- We demonstrate this by using Monte Carlo simulation to compute the
  payoffs across multiple parallel and independent paths
- The `mc` function generates `n` random paths, applies a payoff
  function and then discounts the result
- The `eu` function defines the European option payoff function
- The `mcstat` function returns the expected value, error and count of
  the simulations


In [ ]:
mc:{[S;s;r;t;pf;n]
 z:bm n?/:count[t]#1f;
 f:S*prds gbm[s;r;deltas[first t;t]] z;
 v:pf[f]*exp neg r*last t;
 v}
eu:{[c;k;f]0f|$[c;last[f]-k;k-last f]}
mcstat:{`ev`err`n!(avg x;1.96*sdev[x]%sqrt n;n:count x)}

## Confirming Monte Carlo accuracy
- Comparing the Black-Scholes-Merton model with the Monte Carlo model
  shows an accurate expected value


In [ ]:
bsm[S;k;r;t;c;s]`price 


In [ ]:
mcf:mc[S;s;r;0f,t;eu[c;k]] / declare a mcf projection for reuse later
mcstat mcf 10000000

## Secondary threads
- Starting kdb+ with a `-s` parameter starts the specified number of
  secondary threads
- We can check (and change -- but not within a jupyter notebook) the
  number of secondary processes with the `\s` system command
- PyKX starts with as many secondary enabled as are available (based on the number of cores on the machine and the number of cores allowed by the license.

In [ ]:
\s

## Performance of parallel executions
- Using `peach` instead of `each` allows us to run a function in
  parallel across a list
- Using the `\t` system command we can time the results

In [ ]:
\t 0N!mcstat mcf 1000000              / vectorized
\t 0N!mcstat raze mcf each 10#100000  / eached
\t 0N!mcstat raze mcf peach 10#100000 / peached

# Attacker Success Probability

Satoshi Nakamoto famously coined the **attacker success probability**
in his paper: [Bitcoin: A Peer-to-Peer Electronic Cash
System](https://bitcoin.org/bitcoin.pdf)

$$ P = 1 - \sum_{k=0}^z  \frac{\lambda^ke^{-\lambda}}{k!} \left(1-(q/p)^{(z-k)}\right) $$

where:

- $p$ is the probability an honest node finds the next block
- $q$ is the probability the attacker finds the next block
- $z$ is the number of blocks behind the attacker finds themselves
- $ \lambda = z\frac{q}{p} $ (poisson parameter)
- $P$ is the probability an attacker surpasses the honest miners

Note that [A. Pinar Ozisik and Brian Neil
Levine](https://arxiv.org/abs/1701.03977) use Monte Carlo simulation
to determine that part of the function is not perfect (hint: it is the
Poisson density function).


## Attacker Success Probability implementation
- Sourced from Satoshi Nakamoto's paper
```c
#include <math.h>
double AttackerSuccessProbability(double q, int z)
{
    double p = 1.0 - q;
    double lambda = z * (q / p);
    double sum = 1.0;
    int i, k;
    for (k = 0; k <= z; k++)
    {
        double poisson = exp(-lambda);
        for (i = 1; i <= k; i++)
            poisson *= lambda / i;
        sum -= poisson * (1 - pow(q / p, z - k));
    }
    return sum;
}
```

In [ ]:
/ attacker success probability
asp:{[q;z]
 poisson:exp neg lambda:z*qop:q%1f-q;
 p:(prd lambda%1+til::) each k:til 1+z;
 p:1f-poisson*sum p*1f-qop xexp z-k;
 p}

## Probability of attack as a function of z
- The probability drops off exponentially with z
- Assuming honest nodes have 90% probability and dishonest attacker
  has 10% probability


In [ ]:
\c 20 100
P:asp[q:.1] each z:til 11
([q;z]P)

## An attacker with more skill
- Now we assume honest nodes have 70% probability and dishonest
  attacker has 30% probability

In [ ]:
P:asp[q:.3] each z:5*til 11
([q;z]P)

## Number of blocks as a function of P and q
- We can use iteration to find z (the number of blocks behind) that is
  required for a given q (success rate an attacker finds the next
  block) to make the probability of a successful attack .1%
- Iterating with a function as the continuation predicate allows us to
  stop when we've reached the limit


In [ ]:
f:{[P;q](P<asp[q]::) (1+)/ 0} / function to increment z until P>=asp[q;z]
z:f[P:.001] each q:.05*2+til 8
([P;q]z)